In [1]:
import sqlite3
import os

In [2]:
db_path = "crypto.db"

if os.path.exists(db_path):
    os.remove(db_path)
    print("Database file deleted.")
else:
    print("Database file does not exist.")
# Create or connect to a database file
conn = sqlite3.connect(db_path)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

Database file does not exist.


In [3]:
# Create a table
cursor.execute("""
CREATE TABLE IF NOT EXISTS coins (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    symbol TEXT NOT NULL,
    price REAL,
    market_cap REAL,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# Save (commit) the changes
conn.commit()
print("Table 'coins' created successfully.")

Table 'coins' created successfully.


In [4]:
# Insert single record
cursor.execute("""
INSERT INTO coins (name, symbol, price, market_cap)
VALUES ('Bitcoin', 'BTC', 50000, 950000000000)
""")

# Insert multiple records
coins = [
    ('Ethereum', 'ETH', 3000, 350000000000),
    ('Cardano', 'ADA', 1.5, 50000000000),
    ('Solana', 'SOL', 150, 50000000000)
]

cursor.executemany("""
INSERT INTO coins (name, symbol, price, market_cap)
VALUES (?, ?, ?, ?)
""", coins)

conn.commit()

In [5]:
# Get all coins
cursor.execute("SELECT * FROM coins")
all_coins = cursor.fetchall()  # Get all results as a list of tuples
print("All coins:", all_coins)

# Get one coin
cursor.execute("SELECT * FROM coins WHERE symbol = 'BTC'")
bitcoin = cursor.fetchone()  # Get just the first result
print("Bitcoin data:", bitcoin)

All coins: [(1, 'Bitcoin', 'BTC', 50000.0, 950000000000.0, '2025-08-11 17:53:58'), (2, 'Ethereum', 'ETH', 3000.0, 350000000000.0, '2025-08-11 17:53:59'), (3, 'Cardano', 'ADA', 1.5, 50000000000.0, '2025-08-11 17:53:59'), (4, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-11 17:53:59')]
Bitcoin data: (1, 'Bitcoin', 'BTC', 50000.0, 950000000000.0, '2025-08-11 17:53:58')


In [6]:
# Get specific columns with a condition
cursor.execute("SELECT name, price FROM coins WHERE price > 100")
expensive_coins = cursor.fetchall()
print("Expensive coins:", expensive_coins)

Expensive coins: [('Bitcoin', 50000.0), ('Ethereum', 3000.0), ('Solana', 150.0)]


In [7]:
# Update Bitcoin's price
cursor.execute("UPDATE coins SET price = 51000 WHERE symbol = 'BTC'")
conn.commit()

# Delete cheap coins
cursor.execute("DELETE FROM coins WHERE price < 2")
conn.commit()

In [8]:
# This automatically handles opening and closing the connection
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM coins")
    print(cursor.fetchall())
    # No need to manually close - it's handled by the 'with' block

[(1, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-11 17:53:58'), (2, 'Ethereum', 'ETH', 3000.0, 350000000000.0, '2025-08-11 17:53:59'), (4, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-11 17:53:59')]


In [9]:
import pandas as pd

with sqlite3.connect(db_path) as conn:
    # Read SQL query directly into a DataFrame
    df = pd.read_sql("SELECT * FROM coins", conn)
    
print("DataFrame from SQL:")
print(df.head())

DataFrame from SQL:
   id      name symbol    price    market_cap         last_updated
0   1   Bitcoin    BTC  51000.0  9.500000e+11  2025-08-11 17:53:58
1   2  Ethereum    ETH   3000.0  3.500000e+11  2025-08-11 17:53:59
2   4    Solana    SOL    150.0  5.000000e+10  2025-08-11 17:53:59


In [10]:
# Create a new DataFrame
new_coins = pd.DataFrame({
    'name': ['Polkadot', 'Polygon'],
    'symbol': ['DOT', 'MATIC'],
    'price': [7.5, 0.75],
    'market_cap': [7500000000, 5000000000]
})

with sqlite3.connect(db_path) as conn:
    # Write the DataFrame to a new SQL table
    new_coins.to_sql("new_coins", conn, index=False, if_exists='replace')
    
    # Verify it worked
    df_new = pd.read_sql("SELECT * FROM new_coins", conn)
    print("\nNew coins table:")
    print(df_new)


New coins table:
       name symbol  price  market_cap
0  Polkadot    DOT   7.50  7500000000
1   Polygon  MATIC   0.75  5000000000


In [11]:
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("SELECT * FROM coins", conn)

# Now you can use all pandas operations
print("\nBasic statistics:")
print(df.describe())

print("\nAverage price:", df['price'].mean())
print("Total market cap:", df['market_cap'].sum())

# Filter with pandas
expensive = df[df['price'] > 100]
print("\nExpensive coins:")
print(expensive[['name', 'price']])


Basic statistics:
             id         price    market_cap
count  3.000000      3.000000  3.000000e+00
mean   2.333333  18050.000000  4.500000e+11
std    1.527525  28571.095534  4.582576e+11
min    1.000000    150.000000  5.000000e+10
25%    1.500000   1575.000000  2.000000e+11
50%    2.000000   3000.000000  3.500000e+11
75%    3.000000  27000.000000  6.500000e+11
max    4.000000  51000.000000  9.500000e+11

Average price: 18050.0
Total market cap: 1350000000000.0

Expensive coins:
       name    price
0   Bitcoin  51000.0
1  Ethereum   3000.0
2    Solana    150.0


In [13]:
import sqlite3

db_path = "crypto.db"

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()

    # Create coins table
    cursor.execute("DROP TABLE IF EXISTS coins")
    cursor.execute("""
    CREATE TABLE coins (
        coin_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        symbol TEXT NOT NULL UNIQUE
    )
    """)

    # Create prices table with foreign key
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS prices (
        price_id INTEGER PRIMARY KEY AUTOINCREMENT,
        coin_id INTEGER NOT NULL,
        price REAL NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (coin_id) REFERENCES coins (coin_id)
    )
    """)

    # Insert some data
    cursor.execute("INSERT INTO coins (name, symbol) VALUES ('Bitcoin', 'BTC')")
    cursor.execute("INSERT INTO coins (name, symbol) VALUES ('Ethereum', 'ETH')")

    # Get the auto-generated IDs
    eth_id = cursor.lastrowid  # Gets ETH's ID
    btc_id = eth_id - 1        # Gets BTC's ID (since inserted first)

    # Insert price history
    cursor.executemany("""
    INSERT INTO prices (coin_id, price)
    VALUES (?, ?)
    """, [
        (btc_id, 50000),
        (btc_id, 51000),
        (eth_id, 3000),
        (eth_id, 3100)
    ])

    conn.commit()


In [14]:
- Inner join - keeps only rows that match in both tables
- Left Join - **Keeps all rows from the left table**, and matches from the right table if available.
- Right Join - **Keeps all rows from the right table**, matches from the left if available.
- Full Outer Join - **Keeps all rows from both tables**, matching where possible.
with sqlite3.connect(db_path) as conn:
    # Simple inner join
    df = pd.read_sql("""
    SELECT c.name, c.symbol, p.price, p.date
    FROM coins c
    INNER JOIN prices p ON c.coin_id = p.coin_id
    ORDER BY p.date DESC
    """, conn)
    
print("\nCoin prices with join:")
print(df)

SyntaxError: invalid syntax (1224841649.py, line 1)

In [15]:
# Example of LEFT JOIN (get all coins even if they have no prices)
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("""
    SELECT c.name, c.symbol, p.price, p.date
    FROM coins c
    LEFT JOIN prices p ON c.coin_id = p.coin_id
    """, conn)
    
print("\nAll coins with prices (if available):")
print(df)


All coins with prices (if available):
       name symbol    price                 date
0   Bitcoin    BTC  50000.0  2025-08-11 18:20:09
1   Bitcoin    BTC  51000.0  2025-08-11 18:20:09
2  Ethereum    ETH   3000.0  2025-08-11 18:20:09
3  Ethereum    ETH   3100.0  2025-08-11 18:20:09


In [16]:
# Index Frequent Columns to Improve Query Performance
# Lets say you are frequently searching by "symbol" or "name" in the db

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create an index on symbol
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_coins_symbol ON coins(symbol);
""")

# Create an index on name
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_coins_name ON coins(name);
""")

conn.commit()
conn.close()
print("Indexes created.")

Indexes created.


In [17]:
# Use context managers (with statement)

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("""
    SELECT *
    FROM coins c
    """, conn)
# Close connection with commit (if not using context manager)

conn = sqlite3.connect(db_path)
df = pd.read_sql("""
SELECT *
FROM coins
""", conn)
conn.commit() # saving
conn.close() # closing the database connection
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("""
        SELECT *
        FROM coins
        WHERE symbol = 'BTC'
    """, conn)

print(df)

   coin_id     name symbol
0        1  Bitcoin    BTC
